In [11]:
import torch
from project_helper import TweetData
import numpy as np
import pandas as pd
import os

In [12]:
tweet_data = TweetData()

In [14]:
daily_tweets = tweet_data.clean_tweets #[pd.to_datetime(tweet_data.clean_tweets.after4_date)
                                       #<= pd.to_datetime(daily_df.index[-1])]
daily_tweets.after4_date = pd.to_datetime(daily_tweets.after4_date)

In [16]:
daily_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18


In [2]:
model = torch.load('models/trump_rnn_1911.net')

In [3]:
embeddings = model.in_embed.weight.to('cpu').data.numpy()

In [4]:
embeddings.shape

(4574, 100)

In [5]:
arches = os.listdir('models')
arches[0:5]

['trump_rnn_1702.net',
 'trump_rnn_1703.net',
 'trump_rnn_1908.net',
 'trump_rnn_1707.net',
 'trump_rnn_1712.net']

In [6]:
embedding_list = []
for arch in arches:
    model = torch.load('models/{}'.format(arch))
    embedding_list.append(model.in_embed.weight.to('cpu').data.numpy())
    

In [7]:
embedding_list[0].shape

(2576, 100)

In [8]:
data = TweetData('data/opt/trump_archive_db_1911.csv')
words = data.words
vocab_to_int, int_to_vocab = data.vocab_to_int, data.int_to_vocab
int_words = data.int_words

In [33]:
embedding_list[-1]

array([[-0.07983071,  0.13481925, -0.3339518 , ...,  0.07891949,
        -0.03683575, -0.06613905],
       [-0.4210904 ,  0.02706584, -0.1898706 , ..., -0.17446454,
        -0.08358873, -0.03138035],
       [-0.22094908, -0.20108747, -0.04167196, ...,  0.11216594,
         0.00401473,  0.15296404],
       ...,
       [-0.38425526, -0.3069386 ,  0.7174769 , ..., -0.4966805 ,
        -0.22396143, -0.90884376],
       [-1.4828042 ,  0.38311642,  0.23013367, ...,  0.69838697,
         0.10621499,  0.49422497],
       [-0.72725123,  0.19940047, -0.78007627, ...,  0.00616149,
         0.37083137, -0.43986282]], dtype=float32)

In [23]:
daily_tweets.after4_date

timestamp
2019-11-17 19:57:12-06:00     True
2019-11-17 19:56:02-06:00     True
2019-11-17 19:49:47-06:00     True
2019-11-17 19:47:32-06:00     True
2019-11-17 19:30:09-06:00     True
                             ...  
2009-05-12 14:07:28-05:00    False
2009-05-08 20:40:15-05:00    False
2009-05-08 13:38:08-05:00    False
2009-05-05 01:00:10-05:00    False
2009-05-04 18:54:02-05:00    False
Name: after4_date, Length: 28813, dtype: bool

In [48]:
tweet_embeddings = {}
for i, tweet in enumerate(daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]):
    embed_sum = np.zeros(100) 
    tw = tweet.split()
    for word in tw:
        try:
            embed_sum += embedding_list[-1][vocab_to_int[word]]
        except:
            pass
    if len(tw) > 0:    
        tweet_embeddings[i] = embed_sum /len(tw)
    else:
        tweet_embeddings[i] = embed_sum
            
        

In [49]:
len(tweet_embeddings.keys())

9582